Finding scope 3 emissions of a Pharma company.
- Scope 3 category is downstream transportation and distribution
- Data taken from data.world

Data required for calculation of scope 3
1. 

In [2]:
# Import libraries

import pandas as pd

In [4]:
# Collecting data
raw_df = pd.read_csv("./Data/SCMS_Delivery_History_Dataset_20150929.csv")

In [19]:
raw_df['Manufacturing Site'].unique()

array(['Ranbaxy Fine Chemicals LTD', 'Aurobindo Unit III, India',
       'ABBVIE GmbH & Co.KG Wiesbaden', 'Ranbaxy, Paonta Shahib, India',
       'MSD South Granville Australia', "ABBVIE (Abbott) St. P'burg USA",
       'ABBVIE Ludwigshafen Germany', 'Trinity Biotech, Plc',
       'EY Laboratories, USA', 'Cipla, Goa, India', 'BMS Meymac, France',
       'Premier Med. Corp Ltd. India', 'Chembio Diagnostics Sys. Inc.',
       'Inverness Japan', 'Pacific Biotech, Thailand',
       'Standard Diagnostics, Korea', 'GSK Mississauga (Canada)',
       'Gilead(Nycomed) Oranienburg DE', 'Bio-Rad Laboratories',
       'Mylan (formerly Matrix) Nashik', 'Roche Basel', 'GSK Ware (UK)',
       'Novartis Pharma AG, Switzerland', 'BI, Ingelheim, Germany',
       'Not Applicable', 'Ipca Dadra/Nagar Haveli IN', 'MSD, Haarlem, NL',
       'Aspen-OSD, Port Elizabeth, SA', 'ABBVIE (Abbott) Logis. UK',
       'Gland Pharma Ltd Pally Factory', 'GSK Aranda',
       'Hetero Unit III Hyderabad IN', 'ABBVIE (Abbot

In [20]:
raw_df.head()

,ID,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,...,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,C�te d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,30,19,551.0,29.00,0.97,Ranbaxy Fine Chemicals LTD,Yes,13,780.34,NaN
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,240,1000,6200.0,6.20,0.03,"Aurobindo Unit III, India",Yes,358,4521.5,NaN
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,C�te d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,100,500,40000.0,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,171,1653.78,NaN
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,31920,127360.8,3.99,0.07,"Ranbaxy, Paonta Shahib, India",Yes,1855,16007.06,NaN
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,38000,121600.0,3.20,0.05,"Aurobindo Unit III, India",Yes,7590,45450.08,NaN
